In [ ]:
%load_ext autoreload
%autoreload 2

In [ ]:
import scanpy as sc
import sys
sys.path.insert(0, 'D:/GitHub/sample_workflow/')
from HelperFunctions import qc_and_filter, \
                            label_cells, \
                            plot_clusters

In [ ]:
# Dataset id
ds_ids = ['DeZuani2024', 'Miller2023']

# Read in the AnnData files
adatas = []
for ds_id in ds_ids:
    base = f'D:/GitHub/Data/NSCLC/{ds_id}/'
    adata_path = f'{base}{ds_id}_anndata_processed.h5ad'
    adata = sc.read_h5ad(adata_path)
    adatas += [adata]